## This notebook will check the functionality of BP Mushu interface

In the end, it will plot the # of samples within each call of amp.get_data()


Import the library - get the new amp

In [1]:
cd ..

/home/nfcontrol/nf/nf-rtime/mushu


In [2]:
import matplotlib.pyplot as plt

In [3]:
import struct
import json
fhe=open('testsave3.eeg')
fhm=open('testsave3.marker')
fhj=open('testsave3.meta')

### json = header

In [4]:
meta=json.load(fhj)
print(meta)

{'Channels': ['Ch-1', 'Ch-2', 'Ch-3', 'Ch-4', 'Ch-5', 'Ch-6', 'Ch-7', 'Ch-8', 'Ch-9', 'Ch-10', 'Ch-11', 'Ch-12', 'Ch-13', 'Ch-14', 'Ch-15', 'Ch-16', 'Ch-17', 'Ch-18', 'Ch-19', 'Ch-20', 'Ch-21', 'Ch-22', 'Ch-23', 'Ch-24', 'Ch-25', 'Ch-26', 'Ch-27', 'Ch-28', 'Ch-29', 'Ch-30', 'Ch-31', 'Ch-32', 'Ch-33', 'Ch-34', 'Ch-35', 'Ch-36', 'Ch-37', 'Ch-38', 'Ch-39', 'Ch-40', 'Ch-41', 'Ch-42', 'Ch-43', 'Ch-44', 'Ch-45', 'Ch-46', 'Ch-47', 'Ch-48', 'Ch-49', 'Ch-50', 'Ch-51', 'Ch-52', 'Ch-53', 'Ch-54', 'Ch-55', 'Ch-56', 'Ch-57', 'Ch-58', 'Ch-59', 'Ch-60', 'Ch-61', 'Ch-62', 'Ch-63', 'Ch-64'], 'Sampling Frequency': 5000, 'Amp': '<libmushu.driver.brainproducts.BPAmp object at 0x7fd8d024b9b0>'}


In [5]:
meta['Channels']

['Ch-1',
 'Ch-2',
 'Ch-3',
 'Ch-4',
 'Ch-5',
 'Ch-6',
 'Ch-7',
 'Ch-8',
 'Ch-9',
 'Ch-10',
 'Ch-11',
 'Ch-12',
 'Ch-13',
 'Ch-14',
 'Ch-15',
 'Ch-16',
 'Ch-17',
 'Ch-18',
 'Ch-19',
 'Ch-20',
 'Ch-21',
 'Ch-22',
 'Ch-23',
 'Ch-24',
 'Ch-25',
 'Ch-26',
 'Ch-27',
 'Ch-28',
 'Ch-29',
 'Ch-30',
 'Ch-31',
 'Ch-32',
 'Ch-33',
 'Ch-34',
 'Ch-35',
 'Ch-36',
 'Ch-37',
 'Ch-38',
 'Ch-39',
 'Ch-40',
 'Ch-41',
 'Ch-42',
 'Ch-43',
 'Ch-44',
 'Ch-45',
 'Ch-46',
 'Ch-47',
 'Ch-48',
 'Ch-49',
 'Ch-50',
 'Ch-51',
 'Ch-52',
 'Ch-53',
 'Ch-54',
 'Ch-55',
 'Ch-56',
 'Ch-57',
 'Ch-58',
 'Ch-59',
 'Ch-60',
 'Ch-61',
 'Ch-62',
 'Ch-63',
 'Ch-64']

### binary = raw data

In [6]:
import numpy as np

f = open("testsave3.eeg", "r")
m = np.fromfile(f, dtype=np.uint32)

In [7]:
len(m)/64

120400.0

In [8]:
m=m.reshape(64,round(len(m)/64))

In [9]:
m.shape

(64, 120400)

In [10]:
import mne

In [11]:
# Generate some random data

# Initialize an info structure
info = mne.create_info(
    ch_names=meta['Channels'],
    ch_types = ['eeg' for i in range(64)],
    sfreq=meta['Sampling Frequency']
)

custom_raw = mne.io.RawArray(m, info)
print(custom_raw)

Creating RawArray with float64 data, n_channels=64, n_times=120400
    Range : 0 ... 120399 =      0.000 ...    24.080 secs
Ready.
<RawArray  |  None, n_channels x n_times : 64 x 120400 (24.1 sec), ~58.9 MB, data loaded>


### markers

In [12]:
with open('testsave3.marker') as file:
    content = file.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 
print(content)

import re

# create the marker matrix

['1.800000 Sync Off', '3700.700000 Sync Off', '5700.320000 Sync Off', '7800.460000 Sync Off', '9420.040000 Sync Off', '11480.140000 Sync Off', '13620.220000 Sync Off', '15680.320000 Sync Off', '17820.440000 Sync Off', '19900.540000 Sync Off', '21440.080000 Sync Off', '23560.180000 Sync Off']


In [13]:
content

['1.800000 Sync Off',
 '3700.700000 Sync Off',
 '5700.320000 Sync Off',
 '7800.460000 Sync Off',
 '9420.040000 Sync Off',
 '11480.140000 Sync Off',
 '13620.220000 Sync Off',
 '15680.320000 Sync Off',
 '17820.440000 Sync Off',
 '19900.540000 Sync Off',
 '21440.080000 Sync Off',
 '23560.180000 Sync Off']

In [14]:
ev_arr=[]
for i, item in enumerate(content):
    
    out=re.split("[\s]+", item)
    sample = int(float(out[0])/1000*meta['Sampling Frequency'])
    if out[1]=='Sync':
        code = 250
    else:
        code = int(out[1])

    print(out)
    ev_arr.append([sample, 0, code])



# create marker channel for MNE python:
info = mne.create_info(['STI'], custom_raw.info['sfreq'], ['stim'])
stim_data = np.zeros((1, len(custom_raw.times)))
stim_raw = mne.io.RawArray(stim_data, info)
custom_raw.add_channels([stim_raw], force_update_info=True)

# create the marker matrix:
custom_raw.add_events(ev_arr)

['1.800000', 'Sync', 'Off']
['3700.700000', 'Sync', 'Off']
['5700.320000', 'Sync', 'Off']
['7800.460000', 'Sync', 'Off']
['9420.040000', 'Sync', 'Off']
['11480.140000', 'Sync', 'Off']
['13620.220000', 'Sync', 'Off']
['15680.320000', 'Sync', 'Off']
['17820.440000', 'Sync', 'Off']
['19900.540000', 'Sync', 'Off']
['21440.080000', 'Sync', 'Off']
['23560.180000', 'Sync', 'Off']
Creating RawArray with float64 data, n_channels=1, n_times=120400
    Range : 0 ... 120399 =      0.000 ...    24.080 secs
Ready.


In [16]:
%matplotlib qt5

In [ ]:
custom_raw.plot()

In [25]:
custom_raw.save('test2raw.fif')

Writing /home/nfcontrol/nf/nf-rtime/mushu/test2raw.fif
Closing /home/nfcontrol/nf/nf-rtime/mushu/test2raw.fif [done]
